## Scenario

* web 상에서 유저가 특정 앱에 대한 리뷰를 쓴다. 

* 긍정과 부정으로 분류한다. 

* 긍정으로 분류 될 경우 '00을 좋게 평가 해 주셔서 감사합니다.' 메시지 반환. 

* 부정으로 분류 될 경우, 
    * 고객의 리뷰가 위의 키워드 5개 중 어떤 불만에 속하는지 분류 되고, 
    * 반환되는 문장으로 '00의 ** 기능 ** 에 불만이 있으셨군요. 빨리 조치하여 널 만족시켜줄게요' 와 같이 상단의 **키워드**가 포함된 문장이 반환되도록 함
        * keyword =['디자인', '콘텐츠', '기능', '서비스', '정책', '요금', '사양'] 
            * 출처: http://appkorea.org/awardAppraisalInfo.html (대한민국 모바일어워드 앱 평가 기준, UI/UX는 분석상의 이유로 디자인으로 대체)


### TO-DO (Divide & Conquer !!)

* Document Tokenize
* Gensim Lib 이용하기 위해, 이에 맞게 Document 구축
* word2vec 구축 (가중치 매트릭스 구축)
* Term-Document Matrix 구축
* 선정한 7개 키워드만 추출한 가중치 매트릭스와 TDM 내적 
    * 키워드별 Score 도출
    

* 문장을 입력받아, 스코어가 높은 키워드를 리턴

    

In [4]:
import pandas as pd
import numpy as np
import re

### Import Data + Preprocessing

In [5]:
reviews = pd.read_csv('data/dataset.csv', encoding='utf8')

In [5]:
reviews = reviews.drop('_id',axis=1)

In [6]:
neg_reviews = reviews[(reviews['rate'] == 1) | (reviews['rate'] == 2) | (reviews['rate'] == 3) | (reviews['rate'] == 4)]

In [144]:
# def lang_seperation(array):
    
#     only_ko = list() 
    
#     for item in array:
#         text = re.compile('[^ㄱ-ㅣ가-힣]+') 
#         # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
#         result = text.sub(' ', item) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
#         only_ko.append(result)

#         # 정규표현식에 의해 제거되는 부분들을 따로 저장(Punctuation, number, alphabet...)
#         rest = text.findall(item) 
# #         if len(rest) >= 1:
# #             print(rest)
    
#     return only_ko


In [153]:
# rev = list(neg_reviews['review'])
# only_ko = lang_seperation(rev)
# neg_reviews['review'] = only_ko

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
app_ = [text.strip() for text in neg_reviews['app']]
neg_reviews['app'] = app_

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
neg_reviews[100:115]

,app,date,rate,review,user
172,빗썸,2017년 11월 24일,1,최악의 앱입니다. 화면전환시 다운되는건 기본에 각존 편의성도 최악. 거래 불편 돈...,wansook42b
173,빗썸,2017년 11월 24일,1,비트골드인가 들어왔다고 문자오고 팔려고 들어갔더니 접속은 제대로안되고 돈은 계속 사...,김진형
175,빗썸,2017년 11월 27일,1,뭐가 업데이트된건가요? 모바일 구매할때 가격을 누르면 구매수량이 최대치로 설정되서 ...,한승주
176,빗썸,2017년 11월 24일,4,확실히 확실히 어플로 비트코인 거래량 보니까 간편하긴 하네요 . 가끔씩 실행하고 섭...,광현GwangHyeon 이Lee
178,빗썸,2017년 11월 24일,1,아무리 가상화폐지만 큰돈이 거래되고 누군가는 큰수익을 얻는데 이렇게 전산관리하고 나...,jein lee
179,빗썸,2017년 11월 27일,2,차트기능은...? 사용해본 앱 중에서 최악 ㅠㅠ 그래프 분석에 기본적인 이동평균선조...,차준환
182,빗썸,2017년 11월 27일,3,죄송한데 제 투자 손익률 계산해주는 표같은건 없나요? 일일이 계산기로 두드려서 하기...,간장
183,빗썸,2017년 11월 26일,4,일단 보안이 철저하고 사용하기 좋으니까 빗썸 계속 이용중이고요 .. 실시간으로 거...,전영순
184,빗썸,2017년 12월 2일,4,"정말 잘 쓰고 있구요. 건의사항 몇개 요청합니다. 첫째, 보유 코인별 매수평균 단가...",임성민
186,빗썸,2017년 11월 24일,4,다른곳은뭔가 불편하고 복잡해서 쓰기 불편했는데요. 여기는 다르네요 편리해요 ! 괜히...,유다현


In [7]:
docs = neg_reviews['review']

In [8]:
len(docs)

53651

### Tokenize

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from konlpy.tag import Twitter, Mecab, Komoran
from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer

mecab = Mecab()
twitter = Twitter()
komoran = Komoran()

def mecab_tokenizer(text):
    return [t for t in mecab.morphs(text)]

def twit_tokenizer(text):
    return [t for t in twitter.morphs(text, norm=True, stem=True)]

def komoran_tokenizer(text):
    return [t for t in komoran.morphs(text)]

def max_tokenizer(text):
    tokenizer=MaxScoreTokenizer()
    return tokenizer.tokenize(text)

In [11]:
%%time
train_docs = [row for row in docs]
sentences = [max_tokenizer(d) for d in train_docs]

CPU times: user 12.2 s, sys: 78.6 ms, total: 12.3 s
Wall time: 12.3 s


In [12]:
len(train_docs)

53651

In [14]:
train_docs[0]

'오늘 아침 프레이 하면서 페케지를 구매를 했는데 도감 완성 레드젬, 보상회수 레드젬, 페케지 레드젬이 받았다고 하는데 게임에서는 수치가 오르지 않았네요. 블루젬은 잘 들어와 있고요. 게임을 껐다 켜니 수치는 안 오른 그대로고 그 이후 도감 보상은 올라갑니다. 2000레드젬 이상이 안들어왔는데 처리 부탁드립니다.케릭명은 미스테리움 힘센활잡이 입니다.'

In [13]:
len(sentences)

53651

In [15]:
sentences[0]

['오늘',
 '아침',
 '프레이',
 '하면서',
 '페케지를',
 '구매를',
 '했는데',
 '도감',
 '완성',
 '레드젬,',
 '보상회수',
 '레드젬,',
 '페케지',
 '레드젬이',
 '받았다고',
 '하는데',
 '게임에서는',
 '수치가',
 '오르지',
 '않았네요.',
 '블루젬은',
 '잘',
 '들어와',
 '있고요.',
 '게임을',
 '껐다',
 '켜니',
 '수치는',
 '안',
 '오른',
 '그대로고',
 '그',
 '이후',
 '도감',
 '보상은',
 '올라갑니다.',
 '2000레드젬',
 '이상이',
 '안들어왔는데',
 '처리',
 '부탁드립니다.케릭명',
 '은',
 '미스테리움',
 '힘센활잡이',
 '입니다.']

### stopword(for twit_tokenizer)

In [59]:
# def set_stopword(array):
#     stop_w = list()
#     for item in array:
#         for txt in (list(set(item))):
#             if txt in stop_w:
#                 pass
#             else:
#                 stop_w.append(txt)
#     return stop_w

# stopword_kp = [re.findall('[ㄱ-ㅎㅏㅑㅓㅕㅗㅛㅜㅠㅢㅐㅔㅒㅖ]+/KoreanParticle', str(twit_tokenizer(text))) for text in X]
# stop_kp = set_stopword(stopword_kp)
# len(stop_kp)

# stopword_josa = [re.findall('[가-힣]+/Josa', str(twit_tokenizer(text))) for text in X]
# stop_j = set_stopword(stopword_josa)

# stopword_p = [re.findall('[.!?,''~^";]+/Punctuation', str(twit_tokenizer(text))) for text in X]
# stop_p =set_stopword(stopword_p)


# stop_kp.extend(stop_j)
# stop_kp.extend(stop_p)

# stopword = stop_kp
# len(stopword)

0

###  Word2Vec

In [16]:
import gensim
from gensim import models
from gensim import corpora
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models import word2vec, Word2Vec

In [91]:
%%time
model = word2vec.Word2Vec(sentences, size=100, min_count=3, window=3, workers=8, iter=100, sg=1)
model.init_sims(replace=True)

CPU times: user 4min 28s, sys: 1.14 s, total: 4min 29s
Wall time: 56.8 s


In [110]:
model.most_similar(positive='디자인', topn=20)

[('디자인도', 0.5317959785461426),
 ('전보다는', 0.5052891373634338),
 ('로딩속도', 0.5049370527267456),
 ('필터도', 0.49420660734176636),
 ('불안정한지', 0.48502209782600403),
 ('만든놈', 0.48296478390693665),
 ('굿굿굿', 0.4758656620979309),
 ('줌인', 0.4706882834434509),
 ('인터페이스는', 0.4660206437110901),
 ('아니지', 0.4639437198638916),
 ('원하는걸', 0.4600858688354492),
 ('지원하면', 0.45925232768058777),
 ('차이점을', 0.4572221636772156),
 ('모델들이', 0.4571326971054077),
 ('사용중인데', 0.4550212025642395),
 ('촬영할때', 0.45340901613235474),
 ('편리해서', 0.44955873489379883),
 ('2015', 0.4489971697330475),
 ('빠르고', 0.448594331741333),
 ('디자인이', 0.4460482597351074)]

### word_vector matrix

In [93]:
words = list(model.wv.vocab.keys())
word_list = pd.Series(words)

In [94]:
len(word_list)

28232

In [95]:
word_vec = pd.DataFrame(model.syn1neg)

In [96]:
word_vec.set_index(word_list)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
오늘,0.112375,0.068514,0.400275,-0.048557,-0.036596,0.117110,-0.142532,-0.208634,-0.120031,-0.251946,...,-0.304477,0.377950,0.615837,-0.300033,0.094402,-0.037278,-0.083164,0.321508,-0.074027,0.255537
아침,-0.040546,0.161533,0.122740,-0.204846,0.159511,0.258170,0.001138,-0.416950,-0.084273,-0.043915,...,-0.240926,-0.019827,-0.146176,0.107507,-0.030448,-0.252769,0.591010,0.088364,-0.219292,-0.237934
프레이,-0.061717,-0.098399,0.029548,-0.185081,0.219429,0.039535,0.088304,0.093792,-0.128438,-0.175855,...,-0.167458,0.310340,-0.056854,0.292936,0.074621,0.267363,0.403775,0.343729,-0.258591,-0.425838
하면서,-0.122601,-0.011210,0.108756,-0.116501,0.226901,0.222615,0.370823,-0.262105,-0.202231,-0.019392,...,-0.043760,0.142837,0.018869,-0.048623,0.172560,0.085838,-0.264709,0.095662,0.203578,0.102128
구매를,0.018430,0.099132,-0.240360,-0.074895,-0.089038,-0.099216,-0.166679,-0.318568,-0.137089,0.187250,...,-0.021679,-0.028506,0.071073,0.029965,0.276082,-0.211240,0.272856,0.027470,-0.201755,-0.309536
했는데,-0.223713,0.506231,0.270660,-0.101162,0.144084,0.002684,0.216581,-0.114726,-0.213380,0.218518,...,-0.152099,0.127803,0.014811,0.081041,-0.003812,-0.037407,-0.294129,0.147476,-0.276671,-0.398840
도감,0.161148,-0.076271,0.041464,-0.025979,0.069997,-0.197672,-0.267338,0.066846,0.345757,0.520858,...,0.035712,0.333584,0.113890,-0.184273,-0.320949,0.335936,0.467169,0.512207,-0.303023,-0.404905
받았다고,0.170017,0.017637,-0.206105,0.308494,0.493334,-0.327791,0.255475,-0.368362,0.513515,-0.205081,...,0.189786,0.026623,0.399397,0.120395,-0.000970,0.164081,-0.049710,0.288553,0.243012,0.200371
하는데,-0.334718,0.053414,0.092986,0.194553,-0.114048,0.310057,0.087758,-0.193230,0.044198,0.088393,...,-0.375444,-1.265025,0.114401,-0.131985,0.597263,0.030151,-0.212610,-0.089449,-0.286235,-0.474354
수치가,-0.225219,0.145586,-0.100649,0.171969,-0.027050,0.611421,0.242461,-0.302749,0.034813,-0.153833,...,-0.069254,-0.109015,0.102422,0.169951,-0.135650,-0.221507,0.587791,0.074912,-0.280292,-0.165312


In [97]:
len(word_list)

28232

### Distance Matrix

In [ ]:
distances = pd.read_csv('data/distance_matrix.csv')

In [111]:
a = model.syn1neg

In [112]:
%%time
ttm = pd.DataFrame()
for num, i in enumerate(range(10)):
    dists = list()
    for j in range(len(a)):
        dist = np.linalg.norm(a[i]-a[j])
        dists.append(dist)
    
    ttm[words[num]] = dists

CPU times: user 1.86 s, sys: 34.6 ms, total: 1.9 s
Wall time: 1.9 s


In [115]:
test_norm = ttm.set_index(word_list)

### CountVectorizer (TDM)

In [98]:
len(docs)

53651

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

In [100]:
cv = CountVectorizer(tokenizer=max_tokenizer, vocabulary=word_list)

In [101]:
tdm = cv.fit_transform(docs)

In [102]:
tdm.shape

(53651, 28232)

In [103]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.max_tokenizer>,
 'vocabulary': 0            오늘
 1            아침
 2           프레이
 3           하면서
 4           구매를
 5           했는데
 6            도감
 7          받았다고
 8           하는데
 9           수치가
 10          오르지
 11        않았네요.
 12            잘
 13          들어와
 14         있고요.
 15          게임을
 16           껐다
 17           켜니
 18            안
 19           오른
 20         그대로고
 21            그
 22           이후
 23          보상은
 24       올라갑니다.
 25          이상이
 26           처리
 27            은
 28         입니다.
 29           PC
           ...  
 28202    파인드라이브
 28203        온카
 28204      모트라인
 28205       차량의
 28206

In [104]:
print(cv.vocabulary_)

{'오늘': 0, '아침': 1, '프레이': 2, '하면서': 3, '구매를': 4, '했는데': 5, '도감': 6, '받았다고': 7, '하는데': 8, '수치가': 9, '오르지': 10, '않았네요.': 11, '잘': 12, '들어와': 13, '있고요.': 14, '게임을': 15, '껐다': 16, '켜니': 17, '안': 18, '오른': 19, '그대로고': 20, '그': 21, '이후': 22, '보상은': 23, '올라갑니다.': 24, '이상이': 25, '처리': 26, '은': 27, '입니다.': 28, 'PC': 29, '테라': 30, '2년간': 31, '나온다고': 32, '하니': 33, '기대가': 34, '큽니다.': 35, '이름만': 36, '다른': 37, '양산형': 38, '가': 39, '아니라': 40, '진정': 41, '느낄수': 42, '있는': 43, '좋겠네요.': 44, '아직': 45, '해보진': 46, '않았지만': 47, '느낌이': 48, '강하게': 49, '바랍니다.': 50, '오래': 51, '즐길수': 52, '이': 53, '되었으면': 54, '합니다.': 55, '유저를': 56, '위해': 57, '언제나': 58, '컴퓨터로도': 59, '해보지': 60, '않은': 61, '친구들이랑': 62, '나중에': 63, '같이': 64, '레이드': 65, '같은': 66, '것을': 67, '돌': 68, '목적으로': 69, '하고': 70, '있는데요.': 71, '아직까지는': 72, '괜찮지만': 73, '않고': 74, '재미있게': 75, '할': 76, '수': 77, '있도록': 78, '만들어주셨으면': 79, '좋겠어요.': 80, '그러면': 81, '게임은': 82, '제': 83, '기준으로': 84, '최초로': 85, '친구들과': 86, '함께': 87, '한': 88, '될': 89, '것': 90, '같네요.': 91, '진짜': 92,

In [108]:
#TDM(Term-Document 구축)
tdm = tdm.toarray()

In [109]:
tdm.shape

(53651, 28232)

In [147]:
# row: doc, column: vocab
tdm

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [146]:
# row: vocab, column: doc
tdm.T

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
tdm = tdm.T

### Weight Matrix (Using Normal dist.)

In [ ]:
pd.read_csv('data/distance_matrix.csv', encoding='utf8')

In [124]:
pd.DataFrame(test_norm.loc['기능'])

,기능
오늘,6.065698
아침,6.028865
프레이,6.249743
하면서,5.929179
구매를,6.242762
했는데,6.278157
도감,6.405060
받았다고,6.182092
하는데,6.655506
수치가,6.119571


In [126]:
np.array(test_norm)

array([[ 0.        ,  3.04962134,  2.70467591, ...,  3.19131041,
         4.0603261 ,  2.78598571],
       [ 3.04962134,  0.        ,  2.7677846 , ...,  3.17634654,
         3.95856309,  2.52702093],
       [ 2.70467591,  2.7677846 ,  0.        , ...,  3.15027905,
         3.95318246,  2.81852174],
       ..., 
       [ 4.74634552,  4.37174845,  4.82004833, ...,  4.82355642,
         5.22947931,  4.63763809],
       [ 4.49334192,  4.59695816,  4.42819309, ...,  4.94142914,
         5.13805008,  4.82179928],
       [ 4.75038099,  4.08648109,  4.66865492, ...,  4.21680832,
         4.73996067,  4.357059  ]])

In [128]:
std = np.std(np.array(test_norm))

In [129]:
scaler = 1/-2*std

In [137]:
weight = np.exp(scaler*test_norm)

In [144]:
weight_mat = weight.loc[['컨텐츠', '디자인', '기능', '사양', '성능', '요금', '정책']]

In [145]:
weight_mat

,오늘,아침,프레이,하면서,구매를,했는데,도감,받았다고,하는데,수치가
컨텐츠,0.121375,0.115842,0.126324,0.117607,0.139947,0.114655,0.110114,0.105809,0.121891,0.123067
디자인,0.216692,0.242277,0.226421,0.269439,0.224503,0.234520,0.203126,0.242428,0.186108,0.237416
기능,0.131914,0.133546,0.124050,0.138067,0.124340,0.122879,0.117780,0.126885,0.108330,0.129562
사양,0.163936,0.202404,0.193621,0.167114,0.180538,0.173869,0.146673,0.147114,0.145969,0.171541
성능,0.184083,0.164515,0.178513,0.177386,0.166618,0.170594,0.184809,0.144715,0.139996,0.162380
요금,0.201506,0.207757,0.187750,0.199923,0.180202,0.229312,0.170785,0.186270,0.157235,0.192834
정책,0.122675,0.116807,0.118165,0.123897,0.120460,0.123539,0.109455,0.120313,0.114086,0.124698
